# <font color='violet'> Complete Feature Engineering. Assign Ratings to Psychedelic Experience Reports & Analyze Findings
    
Drawing from scraped psychedelic experience reports that were cleaned here: https://github.com/fractaldatalearning/psychedelic_efficacy/blob/main/notebooks/9-kl-reports-clean-engineer.ipynb

In [ ]:
import pandas as pd
import pickle

In [ ]:
# prepare to add local python functions; import modules from src directory
import sys
src = '../src'
sys.path.append(src)

# import local functions
from nlp.feature_engineer import analyze_sentiment

In [ ]:
# Import data
df = pd.read_csv('../data/processed/erowid_cleaned.csv')
df.info()

In [ ]:
# Delete irrelevant columns

In [ ]:
# Prepare for modeling

<font color='violet'> Use pre-trained model to assign ratings

In [ ]:
# load pickled trained classifier with which to assign ratings
with open("../models/trained_model.pkl", "rb") as f:
    model = pickle.load(f)

# Assign ratings
ratings = model.predict(X_unseen)

In [ ]:
df.to_csv('../data/processed/name_this_data.csv')